In [1]:
@file:Repository("*mavenLocal")

%useLatestDescriptors on

%use kalasim(0.7.95)

import org.kalasim.*
import org.kalasim.plot.kravis.display
import kotlin.math.max
import kotlin.math.roundToInt


In [2]:
// :classpath

Notebook implementatio blocked by https://github.com/Kotlin/kotlin-jupyter/issues/355

In [7]:

sealed class ShiftID

object A : ShiftID()
object B : ShiftID()
object WeekEnd : ShiftID()

val shiftModel = sequence {
    while(true) {
        repeat(5) { yield(A); yield(B) }
        yield(WeekEnd)
    }
}


// model requests with static duration for now once they got hold of an operator
class Request : Component() {
    val callCenter = get<Resource>()

    override fun process() = sequence {
        request(callCenter, capacityLimitMode = CapacityLimitMode.SCHEDULE) {
            hold(1)
        }
    }
}

open class ShiftManager : Component() {
    val shiftIt = shiftModel.iterator()
    val callCenter = get<Resource>()

    override fun repeatedProcess() = sequence {
        val currentShift = shiftIt.next()

        log("starting new shift ${currentShift}")

        // adjust shift capacity at the beginning of the shift
        callCenter.capacity = when(currentShift) {
            A -> 2.0
            B -> 8.0
            WeekEnd -> 0.0
            else -> TODO()
        }

        // wait for end of shift
        hold(if(currentShift == WeekEnd) 48 else 12)
    }
}

abstract class CallCenter(val arrivalRate: Double = 0.3, logEvents: Boolean = true) : Environment(logEvents) {

    // not defined at this point
    abstract val shiftManager : ShiftManager

    val callCenter = dependency { Resource("Call Center") }

    init{
        ComponentGenerator(iat = exponential(arrivalRate)){ Request() }
    }
}

class InterruptingShiftManager: ShiftManager() {
    override fun repeatedProcess() = sequence {
        val currentShift = shiftIt.next()

        log("starting new shift $currentShift")

        // adjust shift capacity at the beginning of the shift
        callCenter.capacity = when(currentShift) {
            A -> 2.0
            B -> 8.0
            WeekEnd -> 0.0
            else -> TODO()
        }

        // complete hangover calls from previous shift
        fun shiftLegacy() = callCenter.claimers.components.filter { it.isInterrupted }

        // incrementally resume interrupted tasks while respecting new capacity
        while(shiftLegacy().isNotEmpty() && callCenter.capacity > 0) {
            val numRunning = callCenter.claimers.components.count { it.isScheduled }
            val spareCapacity = max(0, callCenter.capacity.roundToInt() - numRunning)

            // resume interrupted tasks from last shift to max out new capacity
            shiftLegacy().take(spareCapacity).forEach { it.resume() }

            standby()
        }

        // wait for end of shift
        hold(if(currentShift == WeekEnd) 48 else 12)

        // stop and reschedule the ongoing tasks
        callCenter.claimers.components.forEach {
            // detect remaining task time and request this with high prio so
            // that these tasks are picked up next in the upcoming shift
            it.interrupt()
        }
    }
}


In [ ]:
class SimpleCenter: CallCenter() {
    override val shiftManager = ShiftManager()
}

val sim = SimpleCenter()


In [ ]:
sim.run(1000)